# Implementing DeCov Regularizer 

In [ ]:
# https://machinelearningmastery.com/introduction-to-tensors-for-machine-learning/
# https://keras.io/backend/
from keras import backend as K

class Regularizer(object):
    """Regularizer base class.
    """

    def __call__(self, x):
        return 0.

    @classmethod
    def from_config(cls, config):
        return cls(**config)



class DeCovRegularizer(Regularizer):
batch_size= 10 
    def __call__(self, x):
        
        node_numbers=x.shape[1] # in our example it will be 128
        # x will be the output of the layer: (batch_size,n_nodes)
        # means=[u1,u2,u3,u4,....] : where u1 is the (sum of node1 over the batch size)/batch_size
        means=(K.sum(x,axis=0)/batch_size)  #axis=0 to sum over the column not the rows 
        sub_mean=x-means # we subtract the mean from the nodes; for Example, node1-u1, node2-u2 ...... 
        # Constructing the C matrix; however it will be 1Dim=[c1_2,c1_3....c1_n,c2_1,c2_3......,c2_d.......]
        C=K.variable(np.array([0])) # initialize C with 0 --> C=[0]

        for i in range (0,node_numbers):
            for j in range(0,node_numbers):
                if(i !=j): 
                    C_i_j=K.sum(sub_mean[:,i]*sub_mean[:,j],keepdims=True)/batch_size
                    C=K.concatenate([C, C_i_j], axis=-1)

        DecovLoss=K.sum(C*C)/0.5
        return DecovLoss



In [ ]:
from keras.layers import  Dense
from keras.models import Sequential
# here is how you add the regularizer to a layer 

model = Sequential()
model.add(Dense(32, input_dim=784, activity_regularizer=DeCovRegularizer()))

